<a href="https://colab.research.google.com/github/Mustafa223/NN-and-Deep-learning-course/blob/main/Indoors_Outdoors_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tflearn

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import cv2 as cv
from builtins import input
import argparse
from random import shuffle
from tqdm import tqdm
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf
from google.colab.patches import cv2_imshow

In [ ]:
!pip install -q kaggle 

from google.colab import files

files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

Saving kaggle.json to kaggle.json
ref                                                           title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ruchi798/data-science-job-salaries                            Data Science Job Salaries                           7KB  2022-06-15 08:59:12          13411        432  1.0              
neelgajare/dogecoin-historical-price-data                     DogeCoin Historical Price Data                     34KB  2022-07-28 15:37:35            823         24  1.0              
himanshunakrani/student-study-hours                           Student Study Hours                                276B  2022-07-20 13:17:29            818         32  1.0              
zzettrkalpakbal/full-filled-brain-stroke-datas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!kaggle competitions download -c cv-2022-project-scene-understanding

 99% 927M/933M [00:07<00:00, 129MB/s]
100% 933M/933M [00:07<00:00, 132MB/s]


In [ ]:
!unzip "/content/cv-2022-project-scene-understanding.zip" -d "/content/CV-Dataset"

In [ ]:
tf.__version__

'2.8.2'

In [ ]:
MODEL_NAME = 'Scene Understanding Classification'
gamma = 1.5
IMG_SIZE = 75
Learning_Rate = 1e-3
Train_Names = []
Test_Names = []
imgs = {}
test_imgs = {}
lbls = {}
test_lbls = {}


In [ ]:
def img_estim(img, thrshld):
    is_light = (np.mean(img) > thrshld)
    if is_light==True:
      return 'light'
    else:
      return 'dark'
    # return 'light' if is_light else 'dark'


In [ ]:
def adjust_gamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)


**Read Training Data**

In [ ]:
dir = '/content/CV-Dataset/Train-20220409T231819Z-001/Train/Images'
for img in os.listdir(dir):
  path = os.path.join(dir,img)
  Name = img.split('.')[0]
  imgData = cv2.imread(path)
  if img_estim(imgData,64)=='dark':
    imgData = adjust_gamma(imgData, gamma=gamma)
  Train_Names.append(Name)
  imgs[Name] = imgData

In [ ]:
len(Train_Names)

3000

In [ ]:
TRAIN_IMAGE_LABELS =pd.read_csv('/content/CV-Dataset/TrainImageLabels.csv')
TRAIN_IMAGE_LABELS.head()

,ImageName,LightType,In/Out
0,2019_00001.png,2,1
1,2019_00002.png,6,2
2,2019_00003.png,5,2
3,2019_00004.jpg,3,2
4,2019_00005.jpg,6,2


In [ ]:
TRAIN_IMAGE_LABELS['ImageName'].head()

0    2019_00001.png
1    2019_00002.png
2    2019_00003.png
3    2019_00004.jpg
4    2019_00005.jpg
Name: ImageName, dtype: object

In [ ]:
TRAIN_IMAGE_LABELS['In/Out'].head()

0    1
1    2
2    2
3    2
4    2
Name: In/Out, dtype: int64

In [ ]:
for i in range(len(TRAIN_IMAGE_LABELS['ImageName'])):
  img_name = TRAIN_IMAGE_LABELS['ImageName'][i].split('.')[0]
  img_lbl = TRAIN_IMAGE_LABELS['In/Out'][i]
  print(img_name,' ',img_lbl)
  lbls[img_name] = img_lbl

2019_00001   1
2019_00002   2
2019_00003   2
2019_00004   2
2019_00005   2
2019_00006   2
2019_00007   2
2019_00008   2
2019_00009   2
2019_00010   2
2019_00011   2
2019_00012   2
2019_00013   2
2019_00014   2
2019_00015   2
2019_00016   2
2019_00017   2
2019_00018   2
2019_00019   2
2019_00020   2
2019_00021   2
2019_00022   2
2019_00023   2
2019_00024   2
2019_00025   2
2019_00026   2
2019_00027   2
2019_00028   2
2019_00029   2
2019_00030   2
2019_00031   2
2019_00032   2
2019_00033   2
2019_00034   2
2019_00035   2
2019_00036   2
2019_00037   2
2019_00038   2
2019_00039   2
2019_00040   2
2019_00041   2
2019_00042   1
2019_00043   2
2019_00044   2
2019_00045   2
2019_00046   2
2019_00047   2
2019_00048   2
2019_00049   2
2019_00050   1
2019_00051   2
2019_00052   2
2019_00053   1
2019_00054   2
2019_00055   2
2019_00056   2
2019_00057   2
2019_00058   2
2019_00059   2
2019_00060   2
2019_00061   2
2019_00062   2
2019_00063   2
2019_00064   2
2019_00065   2
2019_00066   1
2019_00067

In [ ]:
def label_image(lb):
  if lb == 1:
    return [1, 0]
  elif lb == 2:
    return [0, 1]

In [ ]:
label = label_image(lbls['2019_01572'])
print(label)


[1, 0]


In [ ]:
def Create_Train_Data():
  training_data = []
  for name in tqdm(Train_Names):
    label = label_image(lbls[name])
    image = cv2.cvtColor(imgs[name], cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    training_data.append([np.array(image), np.array(label)])
  shuffle(training_data)
  np.save('train_data.npy',training_data)
  return training_data

In [ ]:
train_data = Create_Train_Data()

100%|██████████| 3000/3000 [00:03<00:00, 763.63it/s]
/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [ ]:
for i in train_data:
  print(i[1])


In [ ]:
tf.compat.v1.reset_default_graph()

In [ ]:
convnet = input_data(shape =[None, IMG_SIZE, IMG_SIZE, 1], name ='input')
 
convnet = conv_2d(convnet, 32, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 64, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 128, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 64, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 32, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = fully_connected(convnet, 1024, activation ='relu')
convnet = dropout(convnet, 0.8)
 
convnet = fully_connected(convnet, 2, activation ='softmax')
convnet = regression(convnet, optimizer ='adam', learning_rate = Learning_Rate,
      loss ='categorical_crossentropy', name ='targets')
 
model = tflearn.DNN(convnet, tensorboard_dir ='log')


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
train = train_data[:2400]# 75% of data
test = train_data[2400:]# 25% of data

In [ ]:
len(train) 

2400

In [ ]:
len(test) 

600

In [ ]:
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1) 
Y = [i[1] for i in train] 
test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in test]

In [ ]:
model.fit({'input': X}, {'targets': Y}, n_epoch = 10,
    validation_set =({'input': test_x}, {'targets': test_y}),
    snapshot_step = 500, show_metric = True, run_id = MODEL_NAME)
model.save(MODEL_NAME)

Training Step: 379  | total loss: 0.31924 | time: 14.495s
| Adam | epoch: 010 | loss: 0.31924 - acc: 0.8646 -- iter: 2368/2400
Training Step: 380  | total loss: 0.31738 | time: 15.913s
| Adam | epoch: 010 | loss: 0.31738 - acc: 0.8657 | val_loss: 0.74115 - val_acc: 0.7267 -- iter: 2400/2400
--
INFO:tensorflow:/content/Scene Understanding Classification is not in all_model_checkpoint_paths. Manually adding it.


**Next cells for test data**

In [ ]:
dir = '/content/CV-Dataset/Test/Test/Images'
for img in os.listdir(dir):
  path = os.path.join(dir,img)
  Name = img.split('.')[0]
  imgData = cv2.imread(path)
  if img_estim(imgData,64)=='dark':
    imgData = adjust_gamma(imgData, gamma=gamma)
  Test_Names.append(Name)
  test_imgs[Name] = imgData

In [ ]:
for name in Test_Names:
  print(name)

In [ ]:
df2 = pd.read_csv('/content/CV-Dataset/TestImageLabels.csv')
df2.head()

,ImageName,LightType,In/Out
0,2019_00251.jpg,6,2
1,2019_00252.jpg,5,2
2,2019_00253.jpg,5,2
3,2019_00254.jpg,6,2
4,2019_00255.jpg,6,2


In [ ]:
for i in range(len(df2['ImageName'])):
  img_name = df2['ImageName'][i].split('.')[0]
  
  img_lbl = df2['In/Out'][i]
  print(img_name,' ',img_lbl)
  test_lbls[img_name] = img_lbl

In [ ]:
label = test_lbls['2019_00349']
label
label_image(label)

[1, 0]

In [ ]:
def Create_Test_Data():
  testing_data = []
  for name in tqdm(Test_Names):
    label = test_lbls[name]
    image = cv2.cvtColor(test_imgs[name], cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    testing_data.append([np.array(image), np.array(label)])
  shuffle(testing_data)
  np.save('test_data.npy',testing_data)
  return testing_data

In [ ]:
test_data = Create_Test_Data()

100%|██████████| 1800/1800 [00:04<00:00, 414.01it/s]
/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [ ]:
test_data = np.load('test_data.npy',allow_pickle=True)

In [ ]:
correct_cases=0
for img,label in test_data:
  cv2_imshow(img)
  print(label)
  img = img.reshape(IMG_SIZE,IMG_SIZE,1)
  # cv2_imshow(img)
  model_out = model.predict([img])

  if np.argmax(model_out)==1: ans=2
  else: ans=1
  if ans==label:
    correct_cases+=1
  
  print('Y_pred =',ans,'Y_actual = ',label)


In [ ]:
test_acc = (correct_cases/1800)*100

In [ ]:
print(test_acc)